In [1]:
#pip install pyproj
#pip install netdcf4
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.spatial.qhull as qhull
from scipy.interpolate import griddata
import pyproj
from u_interpolate_small import regrid_irregular_quick
from datetime import date
from ndays import numOfDays
import glob
#from mpl_toolkits.basemap import Basemap

In [30]:
# Define domain and time period
start_lat = -37
end_lat = -10
start_lon = -18
end_lon = 30
start_year = '2010'
start_month = '12'
end_year = '2011'
end_month = '01'
start_day = '01'
end_day = '31'

In [8]:
# find indices using one file
core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/09/201009081045.nc'
core_data = nc.Dataset(core_filename,'r')
# read in cores lat/lon
lat = np.array(core_data.variables['lat'][:])
lon = np.array(core_data.variables['lon'][:])
#lon_filter = lon >-181 # acceptable values
#lon_fl = lon[lon_filter] #filtered long
lat_ind = np.where((lat>=start_lat) & (lat<=end_lat))[0]
lon_ind = np.where((lon>= start_lon) & (lon<=end_lon))[0]
lat = lat[lat_ind[0]:lat_ind[-1]]
lon = lon[lon_ind[0]:lon_ind[-1]]
#lon = lon_fl[(lon_fl>= start_lon) & (lon_fl<=end_lon)]
X, Y = np.meshgrid(lon,lat)

# find ndays
date1 = date(int(start_year), int(start_month), int(start_day))
date2 = date(int(end_year), int(end_month), int(end_day))
t = numOfDays(date1, date2)
#time = 
cores = np.zeros((len(lat),len(lon),t*24),dtype=float) #using every fourth value for hourly comparison- 24hr
#time = np.zeros((len(lat)-1,len(lon)-1,t*96))

In [7]:
#dir_name = '/prj/Africa_cloud/ch9_wavelet/'+start_year+'/'+start_month+'/'
#os.chdir(dir_name)

In [92]:
# read in cores data
counter = 0
list_of_files = []

if int(start_year)==int(end_year): 
    for m in range(int(start_month),int(end_month)+1,1): # 1 to include the end month
        if m<10:
            month = str(m).zfill(1)
        else:
            month = str(m)               
        dir_name = '/prj/Africa_cloud/ch9_wavelet/'+start_year+'/'+month+'/'       
        all_file_names=sorted(glob.glob(dir_name+start_year+month+"*.nc"));
            #print(all_files)
        list_of_files.append(all_file_names) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        #months_array = np.array(range(int(start_month),12+1,1))
        #months_array= np.append(months_array,range(1,int(end_month)+1,1))
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                dir_name = '/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'       
                all_file_names=sorted(glob.glob(dir_name+str(i)+month+"*.nc"));
                #print(all_file_names)
                list_of_files.append(all_file_names) # all days in month  
                y=y+1
                month=[]
            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)    
                dir_name = '/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'       
                all_file_names2=sorted(glob.glob(dir_name+str(i)+month+"*.nc"));
                list_of_files.append(all_file_names2) # all days in month 
                
                
#for a in len(list_of_files):
 #   list_of_files = list_of_files+list_of_files[a]
list_of_files = list_of_files[0]+list_of_files[1]
print((list_of_files))

['/prj/Africa_cloud/ch9_wavelet/2010/12/201012010000.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010015.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010030.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010045.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010100.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010115.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010130.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010145.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010200.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010215.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010230.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010245.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010300.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010315.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010330.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010345.nc', '/prj/Africa_cloud/ch9_wavelet/2010/12/201012010400.nc', '/prj/Africa_cloud/ch9_wavelet

In [93]:
#list_of_files = sorted(os.listdir())
for l in range(2, min(t*96,len(list_of_files)), 4):#range(len(list_of_files)): # 96 files per day
    core_filename = list_of_files[l]
    print(core_filename)
#    core_filename = '/prj/Africa_cloud/ch9_wavelet/'+start_year+'/'+start_month+'/'+list_of_files[l]
    core_data = nc.Dataset(core_filename,'r')
    #print(core_data)
    cores_temp = np.array(core_data.variables['cores'])[0,lat_ind[0]:lat_ind[-1],lon_ind[0]:lon_ind[-1]]
    cores[:,:,counter] = cores_temp   
    counter = counter+1
    core_data.close()           

/prj/Africa_cloud/ch9_wavelet/2010/12/201012010030.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010130.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010230.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010330.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010430.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010530.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010630.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010730.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010830.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012010930.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011030.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011130.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011230.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011330.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011430.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011530.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011630.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012011730.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2010/12/201012071045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071445.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071545.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012071945.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012072045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012072145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012072245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012072345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012080045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012080145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012080245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012080345.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2010/12/201012132145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012132245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012132345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140445.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140545.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012140945.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012141045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012141145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012141245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012141345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012141445.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2010/12/201012200645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012200745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012200845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012200945.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201445.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201545.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012201945.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012202045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012202145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012202245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012202345.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2010/12/201012261645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012261745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012261845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012261945.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012262045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012262145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012262245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012262345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270045.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270145.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270245.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270345.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270445.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270545.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270645.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270745.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270845.nc
/prj/Africa_cloud/ch9_wavelet/2010/12/201012270945.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2011/01/201101020145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101020945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101021845.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2011/01/201101081045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101081945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101082045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101082145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101082245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101082345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101090045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101090145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101090245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101090345.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2011/01/201101141845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101141945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101142045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101142145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101142245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101142345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101150945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101151045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101151145.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2011/01/201101210245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101210945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101211945.nc
/prj/Africa_cloud/ch9_wavele

/prj/Africa_cloud/ch9_wavelet/2011/01/201101271145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271445.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271545.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271645.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271745.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271845.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101271945.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101272045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101272145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101272245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101272345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101280045.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101280145.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101280245.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101280345.nc
/prj/Africa_cloud/ch9_wavelet/2011/01/201101280445.nc
/prj/Africa_cloud/ch9_wavele

In [68]:
# find precip indices using one file
precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/20100101.0000_20110101.0000/mcstrack_20100901_0630.nc'
precip_data = nc.Dataset(precip_filename,'r')
lat_p = np.array(precip_data.variables['latitude'][:])
lon_p = np.array(precip_data.variables['longitude'][:])
lat_ind_p = np.where((lat_p[:,1]>=start_lat) & (lat_p[:,1]<=end_lat))[0]
lon_ind_p = np.where((lon_p[1,:]>= start_lon) & (lon_p[1,:]<=end_lon))[0]
lat_p = lat_p[lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
lon_p = lon_p[lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
#X_p, Y_p = np.meshgrid(lon_p,lat_p)

In [89]:
# read in precip data 
list_of_files_precip=[]

if int(start_year)==int(end_year): 
    dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+start_year+'0101.0000_'+str(int(start_year)+1)+'0101.0000'
    os.chdir(dir_name_precip)
    for m in range(int(start_month),int(end_month)+1,1):
            if m<10:
                month = str(m).zfill(1)
            else:
                month = str(m)   
            all_files=sorted(glob.glob("mcstrack_"+start_year+month+"*.nc"));
            #print(all_files)
            list_of_files_precip.append(all_files) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
        #os.chdir(dir_name_precip)
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                list_of_files_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                y=y+1            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m) 
                dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
                list_of_files_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                    
    
    #print(dir_name_precip)
list_of_files_precip = list_of_files_precip[0]+list_of_files_precip[1]

In [ ]:
# prespecify
precip = np.zeros((len(lat_p[:,1]),len(lon_p[1,:]),t*24),dtype=float)
regidded_precip = np.zeros((len(lat),len(lon),t*24),dtype=float)
     
for l in range(0,len(list_of_files_precip),1):
    precip_filename = list_of_files_precip[l]
    print(precip_filename)
    precip_data = nc.Dataset(precip_filename,'r')
    precip_temp = np.array(precip_data.variables['precipitation'])[0,lat_ind_p[0]:lat_ind_p[-1],lon_ind_p[0]:lon_ind_p[-1]]
    precip[:,:,l] = precip_temp                                                                                                                                                            
    A = regrid_irregular_quick(lon_p[1,:],lat_p[:,1],lon,lat,precip_temp)
    regidded_precip[:,:,l] = A   

In [ ]:
# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contourf(X,Y,cores[:,:,0],cmap='jet')
plt.colorbar(label='power')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# visualize core data at inherent grid/resol
#plt.figure(figsize=(10,6))
#plt.contourf(lon_p,lat_p,precip[:,:,1],cmap='jet')#
#plt.colorbar(label='precip [mm/hr]')
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contourf(X,Y,regidded_precip[:,:,0],cmap='jet')#
plt.colorbar(label='precip [mm/hr]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# find rainfall pixels collocated with cores
cores_1D = cores.flatten()
precip_1D = regidded_precip.flatten() # cores and precip have same size and indexing - obs in connis notebook
cores_ind_gt_0 = np.where(cores_1D>0)[0]
precip_collocated_w_cores = precip_1D[cores_ind_gt_0] #dat in connis notebook
#print(precip_collocated_w_cores.shape)
#print(max(precip_collocated_w_cores))

In [ ]:
# calculate frequency of rainfall above a threshold 
# dat = 
# threshold = 30mm/hr

def calc_treshReached(dat, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat))

In [ ]:
# calculate frequency of rainfall above a threshold relative to the same frequency for the full rainfall field (native)

def calc_treshReached_vsObs(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat)) / np.sum(np.array(obs)>thresh)/np.sum(np.isfinite(obs))

In [ ]:
# calculate fraction of caught rainfall pixels above a threshold
# calc_treshMissed in connis script

def calc_treshcaught(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.array(obs)>thresh)

In [ ]:
# calculate frequency of rainfall above a threshold for regions with wavelet power above a given percentile

def percentile(dat, wav, thresh, perc):
    pval = np.percentile(wav, perc)
    dats = np.array(dat)[np.array(wav)>=pval]
    return np.sum(dats>thresh)/np.sum(np.isfinite(dats))

In [ ]:
# fraction of pixels where threshold exceeded
# dat = precip_collocated_w_cores
# obs = precip_1D
# thresh = 15
# treached = calc_treshReached(precip_collocated_w_cores,thresh)
# print(treached)

In [ ]:
# fraction of caught pixels
# tcaught = calc_treshcaught(precip_collocated_w_cores,precip_1D, thresh)
# print(tcaught)

In [ ]:
# fraction of threshold reached in dat vs threshold reached in obs
tfrac = calc_treshReached_vsObs(precip_collocated_w_cores,precip_1D, thresh)
print(tfrac)

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels collocated with cores
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores, intens)
    treached.append(cc)

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')

In [ ]:
# plot frequency of caught pixels with rainfall above a threshold for regions with wavelet power above a given 
# percentile
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = percentile(precip_collocated_w_cores, cores_1D[cores_ind_gt_0], intens, 75)
    treached.append(cc)
    #plotdic.append((treached,dat[1]))
#for pp in plotdic:    
#ax.plot(range(5,50,5), pp[0], label=pp[1])

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability for 75th centile power (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
#ax.ylabel('fraction of rainfall pixels caught above intensity threshold that have >= 75th percentile wavelet power')
#plt.legend()
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'rainP_75p_wpower.jpg')

In [ ]:
# fraction of rainfall pixels caught above intensity threshold to all caught rainfall pixels
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

tcaught = []
for intens in range(5,50,5):   
    cc = calc_treshcaught(precip_collocated_w_cores, precip_1D, intens)
    tcaught.append(cc)

ax.plot(range(5,50,5), tcaught,color='red')
ax.set_title("Retained pixels per intensity (%)")
plt.xlabel('intensity of rainfall [mm/hr]')
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'_missedCases_southernAfricaDomain.jpg')   

In [ ]:
# read cores daily data
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/06/201006081045.nc'
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/',month,'/201006081045.nc'
#core_data = nc.Dataset(core_filename,'r')
#print(core_data)
#cores_temp = np.array(core_data.variables['cores'])
#cores = np.array(core_data.variables['cores'])
#time = core_data.variables['time'][:]

#cores = cores.append(cores_temp)
# make a lat x lon x time dataset for monthly cores
#global_attributes = core_data.__dict__


# read precip data
#precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/'
#precip_data = nc.Dataset(precip_filename,'r')

# visualize core data at inherent grid/resol
#plt.figure(figsize=(10,6))
#plt.contourf(X,Y,np.max(cores,axis=2),cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

# visualize regridded data
# plt.figure(figsize=(10,6))
#plt.contourf(X_new,Y_new,A,cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

In [ ]:
# replace with precip data grid - bring coarse precip data (0.1) to fine core grid (0.04)
#new_lat = np.linspace(min(lat),max(lat),2000)#(lat[2]-lat[1])*2)
#new_lon = np.linspace(min(lon),max(lon),1900)#(lat[2]-lat[1])*2)
#X_new, Y_new = np.meshgrid(new_lon,new_lat)
#A = regrid_irregular_quick(lat_p,lon_p,lon,lat,cores_temp)
#regidded_precip[:,:,l] = A    